# Creating a control non-thrombolysed patient data set with similar characteristics to thrombolysed patients

The aim of this notebook is to create 'control' non-thrombolysed patient data that has similar overall patient characteristics to the thrombolysed group of patients, emulating a clinical trial for thrombolysis.

Non-thrombolysed patients will be selected based using a nearest-neighbour method based on key patient characteristics. 

## Import libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# Turn off warnings
import warnings
warnings.filterwarnings("ignore")

# Set maximum number of rows to display
pd.options.display.max_rows = 100

## Import data

In [2]:
all_data = pd.read_csv('../output/reformatted_data.csv')

# Limit to years 2017 to 2019
mask = (all_data['year'] >= 2017) & (all_data['year'] <= 2019)
data = all_data[mask]

# Limit to infarction stroke
mask = (data['infarction'] == 1)
data = data[mask]

# Limit to arrivals by ambulace
mask = (data['arrive_by_ambulance'] == 1)
data = data[mask]

# Remove patiens who have received thrombectomy
mask = (data['thrombectomy'] == 0)

# Remove patients with no recorded prior disability
mask = data['prior_disability'] >= 0
data = data[mask]

# Remove records with no recorded discharge_disability
mask = data['discharge_disability'] >= 0
data = data[mask]

# Remove records with negative onset_to_arrival_time
mask = data['onset_to_arrival_time'] <= 0
mask =  mask == False
data = data[mask]

Limit to patient clinical characteristics (not arrival timings).

In [3]:
required_cols = [
    'age',
    'male',
    'infarction',
    'thrombolysis',
    'congestive_heart_failure',
    'hypertension',
    'atrial_fibrillation',
    'diabetes',
    'prior_stroke_tia',
    'afib_anticoagulant',
    'prior_disability',
    'stroke_severity',
    'nihss_complete',
    'nihss_arrival_loc',
    'nihss_arrival_loc_questions',
    'nihss_arrival_loc_commands',
    'nihss_arrival_best_gaze',
    'nihss_arrival_visual',
    'nihss_arrival_facial_palsy',
    'nihss_arrival_motor_arm_left',
    'nihss_arrival_motor_arm_right',
    'nihss_arrival_motor_leg_left',
    'nihss_arrival_motor_leg_right',
    'nihss_arrival_limb_ataxia',
    'nihss_arrival_sensory',
    'nihss_arrival_best_language',
    'nihss_arrival_dysarthria',
    'nihss_arrival_extinction_inattention',
    'discharge_destination',
    'death',
    'discharge_disability',
    'disability_6_month'
    ]

In [4]:
#data = data[required_cols]

## Split data by use of thrombolysis

In [5]:
# Split the data into two groups
data_thrombolysis = data[data['thrombolysis'] == 1]
data_no_thrombolysis = data[data['thrombolysis'] == 0]

# Shuffle the data
data_thrombolysis = data_thrombolysis.sample(frac=1, random_state=42)
data_no_thrombolysis = data_no_thrombolysis.sample(frac=1, random_state=42)

## Select the columns to be used for nearest neighbour

In [6]:
nn_cols = [
    'prior_disability',
    'stroke_severity',
    'age',
    'male',
    'congestive_heart_failure',
    'hypertension',
    'atrial_fibrillation',
    'diabetes',
    'prior_stroke_tia',
    'afib_anticoagulant'
]

## Standardise the data

In [7]:
# Concatenate the data
concatenated_data = pd.concat([data_thrombolysis, data_no_thrombolysis])
# Scale the data based on concatenated data
scaler = StandardScaler()
scaler.fit(concatenated_data[nn_cols])
data_thrombolysis_standardised = scaler.transform(data_thrombolysis[nn_cols])
data_no_thrombolysis_standardised = scaler.transform(data_no_thrombolysis[nn_cols])

## Find nearest neighbours to each thrombolysed patients

We will find the required number of nearest neighbour points (matching the size of the thrombolysis data set) by increasing the circle of nearest neighbours until the required number of nearest neighbour points have been found.

In [8]:
required_sample_size = data_thrombolysis.shape[0]
selected_sample_size = 0
selected_ids = []
nearest_neighbour_limit = 0

# Loop until the required sample size is reached
while selected_sample_size < required_sample_size:

    # Increment the nearest neighbour limit
    nearest_neighbour_limit += 1

    # Set up nearest neighbour engine based on no-thrombolysis
    nn = NearestNeighbors(n_neighbors=nearest_neighbour_limit, algorithm='auto').fit(
        data_no_thrombolysis_standardised)
    
    # Get the indices of the nearest neighbours to thrombolysis patients
    distances, indices = nn.kneighbors(data_thrombolysis_standardised)
    indices = pd.Series(indices.reshape(len(indices.flatten())))

    # Get the set of unique indices, combined with previously selected indices
    combined_ids = list(set(indices).union(set(selected_ids)))

    # If the combined indices are less than the required sample size, then
    # select all the combined indices
    if len(combined_ids) < required_sample_size:
        selected_ids = combined_ids
    # Otherwise, sample the required number of new indices
    else:
        # Find unique indices that are not in previously selected indices
        unique_new_ids = list(set(combined_ids) - set(selected_ids))
        number_of_new_ids = required_sample_size - len(selected_ids)
        # Sample the required number of new indices
        new_ids = np.random.choice(unique_new_ids, number_of_new_ids, replace=False)
        # Combine the new indices with previously selected indices
        selected_ids = list(set(selected_ids).union(set(new_ids)))

    # Update the selected sample size
    selected_sample_size = len(selected_ids)

# Select the required data
sampled_no_thrombolysis = data_no_thrombolysis.iloc[selected_ids]
print (f'Last nearest neighbour limit: {nearest_neighbour_limit}')

Last nearest neighbour limit: 3


## Show data statistics

### Summary statistics

In [9]:
results = pd.DataFrame()
results['all_data'] = data.mean()
results['all no thrombolysis'] = data_no_thrombolysis.mean()
results['sampled no thrombolysis'] = sampled_no_thrombolysis.mean()
results['thrombolysis'] = data_thrombolysis.mean()
results

,all_data,all no thrombolysis,sampled no thrombolysis,thrombolysis
id,141158.755990,140895.973278,140549.547817,142542.286853
age,75.687763,76.236050,76.147255,72.801073
male,0.514251,0.507489,0.492499,0.549849
infarction,1.000000,1.000000,1.000000,1.000000
onset_to_arrival_time,2794.409008,3302.049814,5000.071153,121.719033
onset_known,0.708432,0.655358,0.632358,0.987863
precise_onset_known,0.368284,0.284172,0.240129,0.811126
onset_during_sleep,0.145052,0.170852,0.191791,0.009220
arrive_by_ambulance,1.000000,1.000000,1.000000,1.000000
call_to_ambulance_arrival_time,-34.660611,5.816183,-117.443832,-240.435359


## Save data

In [10]:
sampled_data = pd.concat([data_thrombolysis, sampled_no_thrombolysis])
sampled_data = sampled_data.sample(frac=1, random_state=42)    
sampled_data.to_csv('../output/nearest_neighbour_sampled_data.csv', index=False)

## Build logistic regresssion models to test the models

Build binary classification models for two outcomes:

1. Good outcome (mRS 0-2)
2. Bad outcome (mRS 5-6)

### Predict good outcome (mRS 0-2)

In [11]:
X_cols = [
    'age',
    'male',
    'congestive_heart_failure',
    'hypertension',
    'atrial_fibrillation',
    'diabetes',
    'prior_stroke_tia',
    'afib_anticoagulant',
    'prior_disability',
    'stroke_severity',
    'thrombolysis'
    ]

# Split the data into X and y
X = sampled_data[X_cols]
y = sampled_data['discharge_disability'] <= 2

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)

# Predict the test set
y_pred = lr.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print (f'Accuracy: {accuracy:0.2f}')

# Get the model coefficients
model_coefficients = pd.DataFrame(index=X_cols)
model_coefficients['coefficient'] = lr.coef_[0]
model_coefficients['odds ratio'] = np.exp(lr.coef_[0])

# Sort model coefficients by absolute value of coefficient
model_coefficients = model_coefficients.reindex(
    model_coefficients['coefficient'].abs().sort_values(ascending=False).index)

# Print the model coefficients
print ('\nModel coefficients sorted by absolute value of coefficient:\n')
print (model_coefficients)


Accuracy: 0.78

Model coefficients sorted by absolute value of coefficient:

                          coefficient  odds ratio
thrombolysis                 0.704605    2.023048
prior_disability            -0.601070    0.548225
diabetes                    -0.157060    0.854653
stroke_severity             -0.154014    0.857260
atrial_fibrillation         -0.144602    0.865366
prior_stroke_tia             0.117208    1.124354
male                         0.074826    1.077696
afib_anticoagulant           0.065969    1.068194
age                         -0.031600    0.968894
congestive_heart_failure    -0.019315    0.980870
hypertension                 0.000764    1.000764


### Predict bad outcome (mRS 5-6)

In [12]:
X_cols = [
    'age',
    'male',
    'congestive_heart_failure',
    'hypertension',
    'atrial_fibrillation',
    'diabetes',
    'prior_stroke_tia',
    'afib_anticoagulant',
    'prior_disability',
    'stroke_severity',
    'thrombolysis'
    ]

# Split the data into X and y
X = sampled_data[X_cols]
y = sampled_data['discharge_disability'] >= 6

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)

# Predict the test set
y_pred = lr.predict(X_test)
accuracy = np.mean(y_pred == y_test)
print (f'Accuracy: {accuracy:0.2f}')

# Get the model coefficients
model_coefficients = pd.DataFrame(index=X_cols)
model_coefficients['coefficient'] = lr.coef_[0]
model_coefficients['odds ratio'] = np.exp(lr.coef_[0])

# Sort model coefficients by absolute value of coefficient
model_coefficients = model_coefficients.reindex(
    model_coefficients['coefficient'].abs().sort_values(ascending=False).index)

# Print the model coefficients
print ('\nModel coefficients sorted by absolute value of coefficient:\n')
print (model_coefficients)

Accuracy: 0.83

Model coefficients sorted by absolute value of coefficient:

                          coefficient  odds ratio
thrombolysis                -0.439111    0.644609
congestive_heart_failure     0.269897    1.309829
male                         0.248699    1.282356
diabetes                     0.245711    1.278530
prior_stroke_tia            -0.216140    0.805622
atrial_fibrillation          0.189800    1.209008
afib_anticoagulant           0.189367    1.208484
stroke_severity              0.143414    1.154207
prior_disability             0.091094    1.095373
age                          0.043909    1.044888
hypertension                 0.011948    1.012020
